In [1]:
import pandas as pd
import os
from pyspark.sql import SparkSession, functions as F



# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("ETL")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/18 06:36:09 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/09/18 06:36:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/18 06:36:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
output_relative_dir = '../../data/curated/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('consumer', 'merchant', 'transaction', 'sa2'): 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)
        

In [3]:
output_relative_dir = '../../data/nulls&missing_analysis/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('consumer', 'merchant', 'transaction', 'sa2'): 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [4]:
from urllib.request import urlretrieve
import zipfile

# Define the output directory where you want to save the downloaded and extracted files
output_relative_dir = '../../data/raw/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
if not os.path.exists(output_relative_dir + "externaldataset"):
    os.makedirs(output_relative_dir + "externaldataset", exist_ok=True)
    
if not os.path.exists(output_relative_dir + "externaldataset/postcode_shapefiles"):
    os.makedirs(output_relative_dir + "externaldataset/postcode_shapefiles", exist_ok=True)
        
print("Download Start")
    
# URL for the first dataset
url1 = "https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip"
output_file1 = output_relative_dir + "/externaldataset/2021_GCP_SA2_for_AUS_short-header.zip"
urlretrieve(url1, output_file1)
    
# URL for the second dataset
url2 = "https://data.gov.au/data/dataset/6cd8989d-4aca-46b7-b93e-77befcffa0b6/resource/cb659d81-5bd2-41f5-a3d0-67257c9a5893/download/asgs2021codingindexs.zip"
output_file2 = output_relative_dir + "/externaldataset/asgs2021codingindexs.zip"
urlretrieve(url2, output_file2)
    
# URL for the third dataset
    
url3 = "https://www.abs.gov.au/statistics/people/population/regional-population/2021-22/32180DS0003_2001-22r.xlsx"
output_file3 = output_relative_dir + "/externaldataset/SA2_Populations_AUS.xlsx"
urlretrieve(url3, output_file3)
    
print("Download Complete")

# List of files to extract from the ZIP files
files_to_extract1 = [
    "2021Census_G02_AUST_SA2.csv",
]

# file from the second dataset
files_to_extract2 = [
    "2022 Locality to 2021 SA2 Coding Index.csv"    
]

# Path to the ZIP files
zip_file_path1 = output_file1
zip_file_path2 = output_file2

# Destination folder for extraction
destination_folder = "../../data/raw/externaldataset"

# Extract files from the first ZIP file
with zipfile.ZipFile(zip_file_path1, 'r') as zip_ref:
    for file_name in files_to_extract1:
        try:
            zip_ref.extract(f"2021 Census GCP Statistical Area 2 for AUS/{file_name}", destination_folder)
            print(f"Extracted from ZIP 1: {file_name}")
        except KeyError:
            print(f"File not found in ZIP 1: {file_name}")

# Extract files from the second ZIP file
with zipfile.ZipFile(zip_file_path2, 'r') as zip_ref:
    for file_name in files_to_extract2:
        try:
            zip_ref.extract(file_name, destination_folder)
            print(f"Extracted from ZIP 2: {file_name}")
        except KeyError:
            print(f"File not found in ZIP 2: {file_name}")
            

# URL where the zipfile of the shapefiles are stored on the ABS website
zip_file_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/POA_2021_AUST_GDA94_SHP.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))

dir = "data/raw/externaldataset/postcode_shapefiles"

print("Downloading and extracting shapefiles...")

# writing data into directory
z.extractall(dir)

Download Start
Download Complete
Extracted from ZIP 1: 2021Census_G02_AUST_SA2.csv
Extracted from ZIP 2: 2022 Locality to 2021 SA2 Coding Index.csv


## Load in datasets

### Transactions

In [5]:
# 'transactions.parquet' files
path = '../../data/raw/tables/transactions_20210228_20210827_snapshot'
transactions_21_02_21_08 = spark.read.parquet(path)

transactions_21_02_21_08.printSchema()


path = '../../data/raw/tables/transactions_20210828_20220227_snapshot'
transactions_21_08_22_02 = spark.read.parquet(path)

transactions_21_08_22_02.printSchema()

path = '../../data/raw/tables/transactions_20220228_20220828_snapshot'
transactions_22_02_22_08 = spark.read.parquet(path)

transactions_22_02_22_08.printSchema()

transactions_21_02_21_08.limit(10)


root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20
18479,67609108741,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20
18482,70501974849,68.75486276223054,8505fb33-b69a-412...,2021-08-20
4,49891706470,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20


### Consumer Data

In [6]:
# 'consumer_fraud_probability.csv' file
path = '../../data/raw/tables/consumer_fraud_probability.csv'
cust_fp = spark.read.csv(path, header=True)

cust_fp.printSchema()
cust_fp.limit(10)
# cust_fp.dtypes
# cust_fp.count()

root
 |-- user_id: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



user_id,order_datetime,fraud_probability
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035
16556,2022-02-20,89.65663294494827
10278,2021-09-28,83.59136689427714
15790,2021-12-30,71.77065889280253
5233,2021-08-29,85.87123303878818
230,2021-08-28,86.28328808934151


In [7]:
# 'consumer_user_details.parquet' file
path = '../../data/raw/tables/consumer_user_details.parquet'
cust_user_det = spark.read.parquet(path)

cust_user_det.printSchema()
cust_user_det.limit(10)
# cust_user_det.count()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [8]:
# 'tbl_consumer.csv' consumer data file
path = '../../data/raw/tables/tbl_consumer.csv'
cust_tbl = spark.read.csv(path, sep='|', header=True)

cust_tbl.printSchema()
cust_tbl.limit(10)
# cust_tbl.dtypes
# cust_tbl.count()

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: string (nullable = true)



name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


### Merchant Data

In [9]:
# 'merchant_fraud_probability.csv' file
path = '../../data/raw/tables/merchant_fraud_probability.csv'
merch_fp = spark.read.csv(path, header=True)

merch_fp.printSchema()
merch_fp.limit(10)
# merch_fp.dtypes
# merch_fp.count()

root
 |-- merchant_abn: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



merchant_abn,order_datetime,fraud_probability
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902
31334588839,2021-12-26,38.36165958070444
23686790459,2021-12-10,79.4543441508535
14827550074,2021-11-26,46.45775596795885
31334588839,2021-11-26,36.20971272078342
19492220327,2021-12-18,33.819672154331755


In [10]:
# 'tbl_merchants.csv' merchants data file
path = '../../data/raw/tables/tbl_merchants.parquet'
merch_tbl = spark.read.parquet(path)

merch_tbl.printSchema()
merch_tbl.limit(10)

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


### External Data

In [11]:
# 2021 census data CSVs

path = '../../data/raw/externaldataset/2021 Census GCP Statistical Area 2 for AUS/2021Census_G02_AUST_SA2.csv'
sa2_census = spark.read.csv(path, header=True)

sa2_census.limit(10)


SA2_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
101021007,51,1732,760,330,1886,0.8,1429,2.2
101021008,38,1950,975,350,2334,0.8,1989,2.6
101021009,37,1700,996,330,2233,0.9,1703,2.1
101021010,36,1700,1104,310,2412,0.9,1796,2.1
101021012,37,2300,1357,430,3332,0.8,3014,2.9
101021610,31,2600,1629,531,3444,0.8,3363,3.1
101021611,42,2417,1243,465,3117,0.8,2926,2.9
101031013,49,1083,704,200,1759,0.7,1253,2.2
101031014,44,1300,771,290,1911,0.7,1377,2.2
101031015,50,1627,794,290,2068,0.7,1596,2.4


In [12]:
###############################################################################################################################################################################################

In [13]:
# SA2 population data

path = '../../data/raw/externaldataset/SA2_Populations_AUS.xlsx'

# Have to manually fix the dataframe as the formatting is messed up

sa2_pops = pd.read_excel(path, sheet_name=1)

# remove empty / useless rows at top
sa2_pops = sa2_pops.iloc[5:,:]

sa2_pops.head()

# drop unnecessary columns
sa2_pops = sa2_pops.drop(sa2_pops.iloc[:, 10:30],axis = 1)

sa2_pops = sa2_pops.drop(sa2_pops.columns[[0,1,2,3,4,6,11]], axis=1)

# fix header
sa2_pops.columns = sa2_pops.iloc[1]

sa2_pops = sa2_pops[3:]

# drop empty rows at bottom of table
sa2_pops = sa2_pops.dropna()

sa2_pops = sa2_pops.rename(columns={'no.': 'population_2021'})

sa2_pops.head()

6,SA4 name,SA3 name,SA2 code,SA2 name,population_2021
8,Capital Region,Queanbeyan,101021007,Braidwood,4332
9,Capital Region,Queanbeyan,101021008,Karabar,8548
10,Capital Region,Queanbeyan,101021009,Queanbeyan,11375
11,Capital Region,Queanbeyan,101021010,Queanbeyan - East,5097
12,Capital Region,Queanbeyan,101021012,Queanbeyan West - Jerrabomberra,12748


In [14]:
###############################################################################################################################################################################################

In [15]:
# Locality to SA2 coding index CSV

path = '../../data/raw/externaldataset/2022 Locality to 2021 SA2 Coding Index.csv'

sa2_to_postcode = spark.read.csv(path, header=True)

sa2_to_postcode.limit(10)

LOCALITY_ID,LOCALITY_NAME,LOCALITY_TYPE,POSTCODE,STATE,SA2_MAINCODE_2021,SA2_NAME_2021
ABS195,ACTON PARK,EXTRA LOCALITY,2600,ACT,801051123,Black Mountain
loc6f02f098e82d,ACTON,GAZETTED LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,ANU,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,AUSTRALIAN NATION...,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,SPINNAKER ISLAND,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,SPRINGBANK ISLAND,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc8af26acb532d,AINSLIE,GAZETTED LOCALITY,2602,ACT,801051050,Ainslie
locf0cacaa1119e,AMAROO,GAZETTED LOCALITY,2914,ACT,801041034,Amaroo
locf94248966b6c,ARANDA,GAZETTED LOCALITY,2614,ACT,801011001,Aranda
ABS358,BELLVIEW,EXTRA LOCALITY,2620,ACT,801071089,Tuggeranong


## 1st Step: Remove useless columns, data type conversions, rename columns where necessary

### Transactions

In [16]:
#transactions_21_02_21_08 = transactions_21_02_21_08.drop('order_id')

#transactions_21_08_22_02 = transactions_21_08_22_02.drop('order_id')

#transactions_22_02_22_08 = transactions_22_02_22_08.drop('order_id')

transactions_21_02_21_08.printSchema()

transactions_21_08_22_02.printSchema()

transactions_22_02_22_08.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



### Consumer Data

In [17]:
cust_fp = \
cust_fp.withColumn(
    'user_id',
    F.col('user_id').cast('long')
).withColumn(
    'order_datetime',
    F.col('order_datetime').cast('date')
).withColumn(
    'consumer_fraud_probability_%',
    F.col('fraud_probability').cast('double')
).drop('fraud_probability')

cust_fp.printSchema()


root
 |-- user_id: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- consumer_fraud_probability_%: double (nullable = true)



In [18]:
cust_user_det.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



In [19]:
cust_tbl = \
cust_tbl.withColumn(
    'postcode',
    F.col('postcode').cast('long')
).withColumnRenamed(
    'name',
    'consumer_name'   
).withColumnRenamed(
    'state',
    'consumer_state'   
).withColumnRenamed(
    'postcode',
    'consumer_postcode'   
).withColumnRenamed(
    'gender',
    'consumer_gender'   
).withColumn(
    'consumer_id',
    F.col('consumer_id').cast('long')
).drop('address')

cust_tbl.printSchema()

root
 |-- consumer_name: string (nullable = true)
 |-- consumer_state: string (nullable = true)
 |-- consumer_postcode: long (nullable = true)
 |-- consumer_gender: string (nullable = true)
 |-- consumer_id: long (nullable = true)



### Merchant Data

In [20]:
merch_fp = \
merch_fp.withColumn(
    'merchant_abn',
    F.col('merchant_abn').cast('long')
).withColumn(
    'order_datetime',
    F.col('order_datetime').cast('date')
).withColumn(
    'merchant_fraud_probability_%',
    F.col('fraud_probability').cast('double')
).drop('fraud_probability')

merch_fp.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_fraud_probability_%: double (nullable = true)



In [21]:
merch_tbl = \
merch_tbl.withColumnRenamed(
    'name',
    'merchant_name'
)

merch_tbl.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



### External Data

In [22]:
# 2021 census data CSVs

sa2_census.printSchema()


root
 |-- SA2_CODE_2021: string (nullable = true)
 |-- Median_age_persons: string (nullable = true)
 |-- Median_mortgage_repay_monthly: string (nullable = true)
 |-- Median_tot_prsnl_inc_weekly: string (nullable = true)
 |-- Median_rent_weekly: string (nullable = true)
 |-- Median_tot_fam_inc_weekly: string (nullable = true)
 |-- Average_num_psns_per_bedroom: string (nullable = true)
 |-- Median_tot_hhd_inc_weekly: string (nullable = true)
 |-- Average_household_size: string (nullable = true)



In [23]:
sa2_census = sa2_census.withColumn(
    'sa2_code',
    F.col('SA2_CODE_2021').cast('long')
).withColumn(
    'sa2_median_age',
    F.col('Median_age_persons').cast('long')
).withColumn(
    'sa2_median_mortgage_repay_monthly',
    F.col('Median_mortgage_repay_monthly').cast('long')
).withColumn(
    'sa2_median_tot_prsnl_inc_weekly',
    F.col('Median_tot_prsnl_inc_weekly').cast('long')
).withColumn(
    'sa2_median_rent_weekly',
    F.col('Median_rent_weekly').cast('long')
).withColumn(
    'sa2_median_tot_fam_inc_weekly',
    F.col('Median_tot_fam_inc_weekly').cast('long')
).withColumn(
    'sa2_average_num_psns_per_bedroom',
    F.col('Average_num_psns_per_bedroom').cast('double')
).withColumn(
    'sa2_median_tot_hhd_inc_weekly',
    F.col('Median_tot_hhd_inc_weekly').cast('long')
).withColumn(
    'sa2_average_household_size',
    F.col('Average_household_size').cast('double')
)

sa2_census = sa2_census.select(sa2_census.columns[9:])

In [24]:
sa2_census.printSchema()

root
 |-- sa2_code: long (nullable = true)
 |-- sa2_median_age: long (nullable = true)
 |-- sa2_median_mortgage_repay_monthly: long (nullable = true)
 |-- sa2_median_tot_prsnl_inc_weekly: long (nullable = true)
 |-- sa2_median_rent_weekly: long (nullable = true)
 |-- sa2_median_tot_fam_inc_weekly: long (nullable = true)
 |-- sa2_average_num_psns_per_bedroom: double (nullable = true)
 |-- sa2_median_tot_hhd_inc_weekly: long (nullable = true)
 |-- sa2_average_household_size: double (nullable = true)



In [25]:
###############################################################################################################################################################################################

In [26]:
# SA2 population data
sa2_pops=spark.createDataFrame(sa2_pops)

In [27]:
sa2_pops.limit(10)

sa2_pops.printSchema()

root
 |-- SA4 name: string (nullable = true)
 |-- SA3 name: string (nullable = true)
 |-- SA2 code: long (nullable = true)
 |-- SA2 name: string (nullable = true)
 |-- population_2021: long (nullable = true)



In [28]:
# convert casings and rename columns where necessary
sa2_pops = \
sa2_pops.withColumnRenamed(
    'SA4 name',
    'sa4_name'
).withColumnRenamed(
    'SA3 name',
    'sa3_name'
).withColumnRenamed(
    'SA2 code',
    'sa2_code'
).withColumnRenamed(
    'SA2 name',
    'sa2_name'
)


In [29]:
###############################################################################################################################################################################################

In [30]:
# Locality to SA2 coding index CSV
sa2_to_postcode.limit(10)

sa2_to_postcode.printSchema()



root
 |-- LOCALITY_ID: string (nullable = true)
 |-- LOCALITY_NAME : string (nullable = true)
 |-- LOCALITY_TYPE: string (nullable = true)
 |-- POSTCODE: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- SA2_MAINCODE_2021: string (nullable = true)
 |-- SA2_NAME_2021: string (nullable = true)



In [31]:
sa2_to_postcode = sa2_to_postcode.select(['POSTCODE', 'SA2_MAINCODE_2021']).withColumn(
    'POSTCODE',
    F.col('POSTCODE').cast('long')
).withColumn(
    'SA2_MAINCODE_2021',
    F.col('SA2_MAINCODE_2021').cast('long')
).withColumnRenamed(
    'SA2_MAINCODE_2021',
    'sa2_code'
)

sa2_to_postcode = sa2_to_postcode.toDF(*[c.lower() for c in sa2_to_postcode.columns])

## 2nd Step: Cleaning, feature engineering, aggregation

### Transactions

In [32]:
transactions_all = transactions_21_02_21_08.union(transactions_21_08_22_02).union(transactions_22_02_22_08)

In [33]:
transactions_all.sort('order_datetime').head(3)

[Row(user_id=1, merchant_abn=28000487688, dollar_value=133.22689421562643, order_id='0c37b3f7-c7f1-48cb-bcc7-0a58e76608ea', order_datetime=datetime.date(2021, 2, 28)),
 Row(user_id=1, merchant_abn=83690644458, dollar_value=30.441348317517228, order_id='40a2ff69-ea34-4657-8429-df7ca957d6a1', order_datetime=datetime.date(2021, 2, 28)),
 Row(user_id=18485, merchant_abn=62191208634, dollar_value=79.13140006851712, order_id='9e18b913-0465-4fd4-92fd-66d15e65d93c', order_datetime=datetime.date(2021, 2, 28))]

In [34]:
transactions_all.sort('order_datetime').tail(3)

[Row(user_id=11136, merchant_abn=63685007785, dollar_value=48.12988669206644, order_id='a6194c61-ae7a-4037-a425-2dd7cda42fb7', order_datetime=datetime.date(2022, 10, 26)),
 Row(user_id=11137, merchant_abn=85502310765, dollar_value=4.768751754248673, order_id='72a3639e-25d5-4bcc-9e8b-26e737b8c0e4', order_datetime=datetime.date(2022, 10, 26)),
 Row(user_id=11138, merchant_abn=43186523025, dollar_value=35.12766841919401, order_id='3f3b1d40-e8ec-4d4b-ac08-5f273586523c', order_datetime=datetime.date(2022, 10, 26))]

In [35]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = transactions_all

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/transaction/transactions_all.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()
df = df.dropDuplicates(['order_id'])

# 3. Validate user_id and merchant_abn
df = df.filter((F.col('user_id') > 0) & (F.col('merchant_abn') > 0))

# 4. Validate dollar value
df = df.filter(F.col('dollar_value') > 0)
df = df.withColumn('dollar_value', F.round(F.col('dollar_value'), 2))

# 5. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
df = df.filter(F.col('order_datetime').between(start_date, end_date))


transactions_all_clean = df.drop('order_id')

In [36]:
transactions_all_clean

user_id,merchant_abn,dollar_value,order_datetime
83,46804135891,40.89,2021-08-20
18574,40252040480,110.21,2021-08-20
18653,17324645993,6.07,2021-08-20
215,81219314324,66.32,2021-08-20
18860,77013874702,2.08,2021-08-20
18881,29616684420,91.12,2021-08-20
399,71674475255,34.45,2021-08-20
646,24852446429,44.29,2021-08-20
19179,95492562735,43.39,2021-08-20
19230,11437621158,259.73,2021-08-20


### Consumer Data

In [37]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = cust_user_det

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/consumer/consumer_user_details.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate consumer_id and user_id
df = df.filter((F.col('consumer_id') > 0) & (F.col('user_id') > 0))

cust_user_det_clean = df

In [38]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = cust_tbl

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/consumer/consumer_tbl.parquet")


# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate consumer_id
df = df.filter((F.col('consumer_id') > 0))

# 4. Validate consumer_name 
df = df.filter(F.col('consumer_name').rlike("[a-zA-Z][a-zA-Z ]+"))

# 5. Validate consumer_state 
df = df.filter(F.col('consumer_state').isin(['NT','ACT','SA','TAS','WA','QLD','VIC','NSW']))

# 6. Validate consumer_postcode
df = df.filter(F.col('consumer_postcode').between(200, 9729))

# 7. Validate consumer_gender
df = df.filter(F.col('consumer_gender').isin(['Undisclosed', 'Male', 'Female']))

cust_tbl_clean = df
               
            

In [39]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = cust_fp

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/consumer/consumer_fraud_probability.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate user_id
df = df.filter((F.col('user_id') > 0))

# 4. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
df = df.filter(F.col('order_datetime').between(start_date, end_date))

# 5. Validate consumer fraud probability
df = df.filter(F.col('consumer_fraud_probability_%').between(0, 100))
df = df.withColumn('consumer_fraud_probability_%', F.round(F.col('consumer_fraud_probability_%'), 4))

cust_fp_clean = df

In [40]:
cust_fp_clean.sort('user_id')

user_id,order_datetime,consumer_fraud_probability_%
1,2022-02-20,9.8054
2,2021-09-25,10.0699
2,2021-08-30,9.5995
3,2021-11-03,8.3006
4,2021-10-09,9.6333
5,2022-01-11,27.4962
5,2022-02-08,9.0202
5,2021-10-04,10.8684
6,2021-12-12,10.4593
9,2021-12-13,10.5806


### Merchant Data

In [41]:
merch_tbl.count()

4026

In [42]:
# Extract different features from tags (words, revenue band, take rate)
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType, ArrayType

# Define a UDF to extract the sections within square and normal brackets
def split_tags(input_tags):
    sep_tags = re.findall(r'[\[\(\{]([^\[\]\(\)\{\}]*)[\]\)\}]', input_tags)
    sep_tags_without_brackets = [re.sub(r'[\[\]\(\)\{\}]', '', tag) for tag in sep_tags]
        
    return sep_tags_without_brackets

# Define a UDF to extract "take rate" numbers
def extract_take_rate(input_rate):
    match = re.search(r'take rate: ([\d.]+)', input_rate)
    if match:
        return float(match.group(1))
    return None

# Register the UDFs
split_tags_udf = udf(split_tags, ArrayType(StringType()))

clean_take_rate_udf = udf(extract_take_rate, FloatType())

# Apply the UDF to the DataFrame
merch_tbl_clean = merch_tbl.withColumn("sep_tags", split_tags_udf(merch_tbl["tags"]))

# Create separate columns each segment of the tags
merch_tbl_clean = merch_tbl_clean.withColumn("words", merch_tbl_clean["sep_tags"].getItem(0))
merch_tbl_clean = merch_tbl_clean.withColumn("revenue_level", merch_tbl_clean["sep_tags"].getItem(1))
merch_tbl_clean = merch_tbl_clean.withColumn("take_rate_%", merch_tbl_clean["sep_tags"].getItem(2))

merch_tbl_clean = merch_tbl_clean.withColumn("take_rate_%", clean_take_rate_udf(merch_tbl_clean["take_rate_%"]))



In [43]:
merch_tbl_clean = merch_tbl_clean.drop('tags', 'sep_tags')

merch_tbl_clean.limit(10)

merchant_name,merchant_abn,words,revenue_level,take_rate_%
Felis Limited,10023283211,"furniture, home f...",e,0.18
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",b,4.22
Nunc Sed Company,10165489824,"jewelry, watch, c...",b,4.4
Ultricies Digniss...,10187291046,"wAtch, clock, and...",b,3.29
Enim Condimentum PC,10192359162,music shops - mus...,a,6.33
Fusce Company,10206519221,"gift, card, novel...",a,6.34
Aliquam Enim Inco...,10255988167,"computers, comPUt...",b,4.32
Ipsum Primis Ltd,10264435225,"watch, clock, and...",c,2.39
Pede Ultrices Ind...,10279061213,computer programm...,a,5.71
Nunc Inc.,10323485998,"furniture, home f...",a,6.61


In [44]:
print(merch_tbl_clean.select('words').distinct().count())

merch_tbl_clean.select('words').distinct().collect()

971


[Row(words='stationery, office  supplies and printing and writing paper'),
 Row(words='antique shops - sales, repairs, and resToraTion services'),
 Row(words='health  and beauty spas'),
 Row(words='tEnt and awning shops'),
 Row(words='florists supplies, nuRsery sTock, and flowers'),
 Row(words='watcH, clock, and jewelry rEpair shops'),
 Row(words='motor vehiCle supplies and new parts'),
 Row(words='books, periodicaLs,  and newspapers'),
 Row(words='opticians, optical goods, and eyeglasses'),
 Row(words='books, periodicals, and  newspapers'),
 Row(words='cable, satellite, anD other pay telEvision And radio services'),
 Row(words='equipment, tool, furniTure, and appliance rent al and leasiNg'),
 Row(words='gift, carD, novelty, and souvenir shops'),
 Row(words='computers, computer peripheral equipment, and sOftware'),
 Row(words='furniture, home fuRnishings and equipment shops, and manufacturers, except appliances'),
 Row(words='cable, satellite, and other pay television and raDIo service

In [45]:
# Replace multiple spaces with a single space
merch_tbl_clean = merch_tbl_clean.withColumn("words", F.regexp_replace(F.col("words"), "\\s+", " "))

# Trim leading and trailing spaces
merch_tbl_clean = merch_tbl_clean.withColumn("words", F.regexp_replace(F.col("words"), "^\\s+", ""))
merch_tbl_clean = merch_tbl_clean.withColumn("words", F.regexp_replace(F.col("words"), "\\s+$", ""))

# Ensure consistent casing
merch_tbl_clean = merch_tbl_clean.withColumn('words', F.lower(F.col('words')))

In [46]:
print(merch_tbl_clean.select('words').distinct().count())

merch_tbl_clean.select('words').distinct().collect()

25


[Row(words='opticians, optical goods, and eyeglasses'),
 Row(words='watch, clock, and jewelry repair shops'),
 Row(words='computer programming , data processing, and integrated systems design services'),
 Row(words='digital goods: books, movies, music'),
 Row(words='books, periodicals, and newspapers'),
 Row(words='florists supplies, nursery stock, and flowers'),
 Row(words='art dealers and galleries'),
 Row(words='antique shops - sales, repairs, and restoration services'),
 Row(words='gift, card, novelty, and souvenir shops'),
 Row(words='equipment, tool, furniture, and appliance rent al and leasing'),
 Row(words='cable, satellite, and other pay television and radio services'),
 Row(words='tent and awning shops'),
 Row(words='artist supply and craft shops'),
 Row(words='stationery, office supplies and printing and writing paper'),
 Row(words='furniture, home furnishings and equipment shops, and manufacturers, except appliances'),
 Row(words='telecom'),
 Row(words='jewelry, watch, cloc

In [47]:
'''

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')

# Create a UDF (User Defined Function) for lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

lemmatize_udf = udf(lemmatize_text, StringType())

merch_tbl_clean.withColumn("lemmas", lemmatize_udf(F.col("merchant_description"))).select('lemmas').distinct().collect()
'''

'\n\nfrom pyspark.sql.functions import udf\nfrom pyspark.sql.types import StringType\nfrom nltk.stem import WordNetLemmatizer\nimport nltk\n\nnltk.download(\'wordnet\')\n\n# Create a UDF (User Defined Function) for lemmatization\nlemmatizer = WordNetLemmatizer()\n\ndef lemmatize_text(text):\n    words = text.split()\n    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]\n    return \' \'.join(lemmatized_words)\n\nlemmatize_udf = udf(lemmatize_text, StringType())\n\nmerch_tbl_clean.withColumn("lemmas", lemmatize_udf(F.col("merchant_description"))).select(\'lemmas\').distinct().collect()\n'

In [48]:
'''
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer, IDF

# Tokenization
from pyspark.ml.feature import RegexTokenizer

tokenizer = RegexTokenizer(inputCol="text_column", outputCol="words", pattern="\\s+")
wordsData = tokenizer.transform(df)

# Calculate TF (Term Frequency)
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures")
model = cv.fit(wordsData)
featurizedData = model.transform(wordsData)


# Calculate IDF (Inverse Document Frequency)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# Select Relevant Columns
tfidfData = rescaledData.select("text_column", "features")

# Show or Save the Result
tfidfData.show(truncate=False)
# To save the result to a file
# tfidfData.write.save("path/to/save/result", format="parquet")
'''

'\nfrom pyspark.sql import SparkSession\nfrom pyspark.ml.feature import CountVectorizer, IDF\n\n# Tokenization\nfrom pyspark.ml.feature import RegexTokenizer\n\ntokenizer = RegexTokenizer(inputCol="text_column", outputCol="words", pattern="\\s+")\nwordsData = tokenizer.transform(df)\n\n# Calculate TF (Term Frequency)\ncv = CountVectorizer(inputCol="words", outputCol="rawFeatures")\nmodel = cv.fit(wordsData)\nfeaturizedData = model.transform(wordsData)\n\n\n# Calculate IDF (Inverse Document Frequency)\nidf = IDF(inputCol="rawFeatures", outputCol="features")\nidfModel = idf.fit(featurizedData)\nrescaledData = idfModel.transform(featurizedData)\n\n# Select Relevant Columns\ntfidfData = rescaledData.select("text_column", "features")\n\n# Show or Save the Result\ntfidfData.show(truncate=False)\n# To save the result to a file\n# tfidfData.write.save("path/to/save/result", format="parquet")\n'

In [49]:
merch_tbl_clean.count()

4026

In [50]:
df = merch_tbl_clean 

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/merchant/merchant_tbl.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate merchant_abn
df = df.filter((F.col('merchant_abn') > 0))

# 4. Validate merchant_name 
df = df.filter(F.col('merchant_name').rlike("[a-zA-Z][a-zA-Z0-9 ]+"))

# 5. Validate words                # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)
                                   # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 
#df = df.filter(F.col('words').rlike("[a-zA-Z][a-zA-Z0-9 ]+"))

# 6. Validate take_rate
df = df.filter((F.col('take_rate_%').between(0, 100)))

# 7. Validate revenue_level
df = df.filter(F.col('revenue_level').isin(['a', 'b', 'c', 'd', 'e']))


merch_tbl_clean = df


In [51]:
merch_tbl_clean

merchant_name,merchant_abn,words,revenue_level,take_rate_%
Nam Foundation,11437621158,motor vehicle sup...,b,4.53
Aliquam Eros Company,16432242544,"hobby, toy and ga...",b,4.08
Sit Amet Consecte...,17055065679,"computers, comput...",b,3.71
Nec Enim LLP,17614133590,music shops - mus...,c,2.93
Adipiscing Lacus ...,20131680113,"furniture, home f...",b,3.26
Nulla In LLP,49368160590,"computers, comput...",b,4.74
Consectetuer Adip...,64974914166,digital goods: bo...,c,2.12
Vulputate Ullamco...,75475698329,"computers, comput...",a,6.18
Non Ante Consulting,78040474274,telecom,c,2.72
A Sollicitudin Or...,78864123417,"equipment, tool, ...",e,0.36


In [ ]:
from pyspark.sql.types import StringType
# Create the segment feature for each merchant

tech_and_electronics = ["computer", "digital", "television", "telecom"]
retail_and_novelty = ["newspaper", "novelty", "hobby", "shoe", "instruments", "bicycle", "craft","office"]
garden_and_furnishings = ["florists", "furniture", "garden", "tent"]
antiques_and_jewellery = ["galleries", "antique", "jewelry"]
specialized_services = ["health", "motor", "opticians"]


def segment(description):
    for segment, keywords in [("tech_and_electronics", tech_and_electronics),
                               ("retail_and_novelty", retail_and_novelty),
                               ("garden_and_furnishings", garden_and_furnishings),
                               ("antiques_and_jewellery", antiques_and_jewellery),
                               ("specialized_services", specialized_services)]:
        if any(keyword in description for keyword in keywords):
            return segment
    return "other"

segment_udf = F.udf(segment, StringType())

merch_tbl_clean = merch_tbl_clean.withColumn("segment", segment_udf(merch_tbl_clean.words))

In [ ]:
merch_tbl_clean.filter(F.col('segment').isNull())

merchant_name,merchant_abn,words,revenue_level,take_rate_%,segment


In [ ]:
merch_tbl_clean.filter(F.col('segment') == 'other')

In [52]:
df = merch_fp

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/merchant/merchant_fraud_probability.parquet")

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate merchant_abn
df = df.filter((F.col('merchant_abn') > 0))

# 4. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
df = df.filter(F.col('order_datetime').between(start_date, end_date))

# 5. Validate merchant fraud probability
df = df.filter(F.col('merchant_fraud_probability_%').between(0, 100))
df = df.withColumn('merchant_fraud_probability_%', F.round(F.col('merchant_fraud_probability_%'), 4))


merch_fp_clean = df


### External Data

In [126]:
# 2021 census data CSVs

df = sa2_census

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/sa2/sa2_census.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate sa2_code
df = df.filter(F.col('sa2_code') > 0)

# 4. Validate median_age
df = df.filter(F.col('sa2_median_age') >= 0)

# 5. Validate median mortgage repay monthly
df = df.filter(F.col('sa2_median_mortgage_repay_monthly') >= 0)

# 6. Validate median total personal weekly income
df = df.filter(F.col('sa2_median_tot_prsnl_inc_weekly') >= 0)

# 7. Validate median weekly rent
df = df.filter(F.col('sa2_median_rent_weekly') >= 0)

# 8. Validate median total family weekly income
df = df.filter(F.col('sa2_median_tot_fam_inc_weekly') >= 0)

# 9. Validate average number of persons per bedroom
df = df.filter(F.col('sa2_average_num_psns_per_bedroom') >= 0)

# 10. Validate median total household weekly income
df = df.filter(F.col('sa2_median_tot_hhd_inc_weekly') >= 0)

# 11. Validate average household size
df = df.filter(F.col('sa2_average_household_size') >= 0)

sa2_census_clean = df

In [127]:
###############################################################################################################################################################################################

In [128]:
sa2_pops.limit(10)

sa2_pops.printSchema()

root
 |-- sa4_name: string (nullable = true)
 |-- sa3_name: string (nullable = true)
 |-- sa2_code: long (nullable = true)
 |-- sa2_name: string (nullable = true)
 |-- population_2021: long (nullable = true)



In [129]:
df = sa2_pops

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/sa2/sa2_pops.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate population
df = df.filter(F.col('population_2021') >= 0)

sa2_pops_clean = df


In [130]:
###############################################################################################################################################################################################

In [131]:
df = sa2_to_postcode

df.write.mode('overwrite').parquet("../../data/nulls&missing_analysis/sa2/sa2_to_postcode.parquet")

# 1. Remove Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# No need to validate postcode as inner join will be performed with validated postcodes

sa2_to_postcode_clean=df

sa2_to_postcode_clean.sort('postcode').limit(10)



postcode,sa2_code
800,701011002
810,701021029
810,701021026
810,701021027
810,701021016
810,701021023
810,701021018
810,701021010
810,701021024
810,701021013


### Aggregation

In [132]:
output_relative_dir = '../../data/curated/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
   

# now, for each type of data set we will need, we will create the paths
for target_dir in ('consumer', 'merchant'): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)


In [133]:
#cons_combined = 

In [134]:
#cons_combined.limit(5)

In [135]:
#cons_combined.write.parquet("../data/curated/consumer/consumer_combined.parquet")

In [136]:
#merch_combined

In [137]:
#merch_combined.limit(5)

In [138]:
#merch_combined.write.parquet("../data/curated/merchant/merchant_combined.parquet")

In [139]:
orig_combined = cust_tbl_clean.join(cust_user_det_clean, on='consumer_id', how='inner') \
.join(transactions_all_clean, on='user_id', how ='inner') \
.join(cust_fp_clean, on=['user_id', 'order_datetime'], how='left').na.fill(0) \
.join(merch_tbl_clean, on='merchant_abn', how='inner') \
.join(merch_fp_clean, on=['merchant_abn', 'order_datetime'], how='left').na.fill(0)



In [140]:
sa2_combined = sa2_to_postcode_clean.join(sa2_pops_clean, on='sa2_code', how='inner').withColumnRenamed('population_2021','sa2_population').join(sa2_census_clean, on='sa2_code', how='inner')


In [141]:
sa2_combined.limit(5)

sa2_code,postcode,sa4_name,sa3_name,sa2_name,sa2_population,sa2_median_age,sa2_median_mortgage_repay_monthly,sa2_median_tot_prsnl_inc_weekly,sa2_median_rent_weekly,sa2_median_tot_fam_inc_weekly,sa2_average_num_psns_per_bedroom,sa2_median_tot_hhd_inc_weekly,sa2_average_household_size
108041165,2429,Mid North Coast,Port Macquarie,Port Macquarie Su...,5286,50,1668,632,300,1534,0.8,1334,2.5
108041165,2444,Mid North Coast,Port Macquarie,Port Macquarie Su...,5286,50,1668,632,300,1534,0.8,1334,2.5
108041165,2441,Mid North Coast,Port Macquarie,Port Macquarie Su...,5286,50,1668,632,300,1534,0.8,1334,2.5
108041165,2446,Mid North Coast,Port Macquarie,Port Macquarie Su...,5286,50,1668,632,300,1534,0.8,1334,2.5
107031143,2528,Illawarra,Kiama - Shellharbour,Warilla,20500,43,1820,587,350,1484,0.8,1191,2.4


In [142]:
# group SA2 statistics by postcodes by taking averages and medians.

postcode_combined = sa2_combined.groupBy("postcode") \
    .agg(F.avg("sa2_population").alias("consumer_postcode_estimated_population"), \
        F.median("sa2_median_age").alias("consumer_postcode_median_age"), \
        F.median("sa2_median_mortgage_repay_monthly").alias("consumer_postcode_median_mortgage_repay_monthly"), \
        F.median("sa2_median_tot_prsnl_inc_weekly").alias("consumer_postcode_median_totl_prsnal_inc_weekly"), \
        F.median("sa2_median_rent_weekly").alias("consumer_postcode_median_rent_weekly"), \
        F.median("sa2_median_tot_fam_inc_weekly").alias("consumer_postcode_median_tot_fam_inc_weekly"), \
        F.avg("sa2_average_num_psns_per_bedroom").alias("consumer_postcode_avg_num_psns_per_bedroom"), \
        F.median("sa2_median_tot_hhd_inc_weekly").alias("consumer_postcode_median_tot_hhd_inc_weekly"), \
        F.avg("sa2_average_household_size").alias("consumer_postcode_avg_household_size")
    ).withColumnRenamed('postcode', 'consumer_postcode')
    


In [143]:
postcode_combined = postcode_combined.select(*[F.round(c, 2).alias(c) for c in postcode_combined.columns])

postcode_combined = postcode_combined.withColumn("consumer_postcode_estimated_population", F.round(F.col("consumer_postcode_estimated_population")))

In [144]:
postcode_combined.limit(10)

consumer_postcode,consumer_postcode_estimated_population,consumer_postcode_median_age,consumer_postcode_median_mortgage_repay_monthly,consumer_postcode_median_totl_prsnal_inc_weekly,consumer_postcode_median_rent_weekly,consumer_postcode_median_tot_fam_inc_weekly,consumer_postcode_avg_num_psns_per_bedroom,consumer_postcode_median_tot_hhd_inc_weekly,consumer_postcode_avg_household_size
800,7578.0,33.0,1781.0,1236.0,450.0,2403.0,1.0,2151.0,2.0
810,3000.0,35.0,1953.0,957.0,345.0,2400.5,0.92,2083.0,2.74
812,3299.0,36.5,2000.0,986.0,375.0,2313.0,0.77,2003.0,2.33
820,2525.0,34.0,2000.0,1237.0,390.0,2682.0,0.81,2231.0,2.01
822,3615.0,30.0,1040.0,367.5,85.0,820.0,1.26,1325.0,3.16
828,1810.0,33.0,2600.0,1015.0,370.0,2468.0,1.1,2250.0,2.9
829,5181.0,34.0,2275.0,1244.5,425.0,2761.5,0.9,2507.0,2.85
830,3650.0,33.0,1875.0,1011.5,355.0,2229.5,0.95,1937.5,2.61
832,5442.0,30.5,2083.5,1188.0,425.0,2639.5,0.88,2410.5,2.83
834,3362.0,41.0,2383.0,1096.0,370.0,2623.0,0.9,2153.0,2.7


In [145]:
all_combined = orig_combined.join(postcode_combined.withColumnRenamed('postcode','consumer_postcode'), on='consumer_postcode', how='inner')

In [146]:
all_combined = all_combined.withColumnRenamed(
    'words',
    'merchant_description'
).withColumnRenamed(
    'revenue_level',
    'merchant_revenue_level'
).withColumnRenamed(
    'take_rate_%',
    'merchant_take_rate_%'
).withColumnRenamed(
    'dollar_value',
    'transaction_dollar_value_$AUD'
)



In [147]:
all_combined.sort('consumer_fraud_probability_%', ascending=False).limit(5)

consumer_postcode,merchant_abn,order_datetime,user_id,consumer_id,consumer_name,consumer_state,consumer_gender,transaction_dollar_value_$AUD,consumer_fraud_probability_%,merchant_name,merchant_description,merchant_revenue_level,merchant_take_rate_%,merchant_fraud_probability_%,consumer_postcode_estimated_population,consumer_postcode_median_age,consumer_postcode_median_mortgage_repay_monthly,consumer_postcode_median_totl_prsnal_inc_weekly,consumer_postcode_median_rent_weekly,consumer_postcode_median_tot_fam_inc_weekly,consumer_postcode_avg_num_psns_per_bedroom,consumer_postcode_median_tot_hhd_inc_weekly,consumer_postcode_avg_household_size
3186,49549583265,2021-12-19,6228,1242133,Mr. James Norton DDS,VIC,Male,1682.69,97.6298,Luctus Et Incorpo...,tent and awning s...,a,5.69,0.0,23044.0,48.0,3467.0,1265.0,600.0,3808.0,0.8,2718.0,2.4
3186,21025433654,2021-12-19,6228,1242133,Mr. James Norton DDS,VIC,Male,2.74,97.6298,Lorem Foundation,digital goods: bo...,a,6.55,0.0,23044.0,48.0,3467.0,1265.0,600.0,3808.0,0.8,2718.0,2.4
3813,67264251405,2021-10-19,22239,184299,George Mendoza,VIC,Male,51876.64,94.7034,Elit Dictum Eu Fo...,art dealers and g...,a,6.4,18.2109,9940.0,40.0,1950.0,767.0,311.0,2162.0,0.8,1897.0,2.8
3123,73972053940,2021-10-06,8347,1691,Brett Campbell,VIC,Male,13.81,92.9914,Quis Tristique Ac...,tent and awning s...,b,3.88,0.0,15881.0,36.0,2492.0,1228.0,411.0,3203.0,0.9,2228.0,2.2
2805,11149063370,2022-02-25,17900,476053,Angela Fuentes,NSW,Female,42797.35,92.7326,Et Arcu Limited,art dealers and g...,b,4.84,51.0154,5698.0,50.0,1164.0,601.0,210.0,1490.0,0.7,1157.0,2.3


In [148]:
all_combined.limit(5)

consumer_postcode,merchant_abn,order_datetime,user_id,consumer_id,consumer_name,consumer_state,consumer_gender,transaction_dollar_value_$AUD,consumer_fraud_probability_%,merchant_name,merchant_description,merchant_revenue_level,merchant_take_rate_%,merchant_fraud_probability_%,consumer_postcode_estimated_population,consumer_postcode_median_age,consumer_postcode_median_mortgage_repay_monthly,consumer_postcode_median_totl_prsnal_inc_weekly,consumer_postcode_median_rent_weekly,consumer_postcode_median_tot_fam_inc_weekly,consumer_postcode_avg_num_psns_per_bedroom,consumer_postcode_median_tot_hhd_inc_weekly,consumer_postcode_avg_household_size
4606,49505931725,2022-10-13,7,511685,Andrea Jones,QLD,Female,80.56,0.0,Suspendisse Ac As...,digital goods: bo...,b,4.7,0.0,9615.0,47.0,992.0,490.0,180.0,1190.0,0.8,950.0,2.4
4606,80518954462,2021-04-11,7,511685,Andrea Jones,QLD,Female,248.26,0.0,Neque Sed Dictum ...,"computers, comput...",b,3.49,0.0,9615.0,47.0,992.0,490.0,180.0,1190.0,0.8,950.0,2.4
4606,20885454195,2021-06-29,7,511685,Andrea Jones,QLD,Female,106.1,0.0,Pharetra Ut Indus...,"cable, satellite,...",b,4.94,0.0,9615.0,47.0,992.0,490.0,180.0,1190.0,0.8,950.0,2.4
4606,78327477486,2022-05-24,7,511685,Andrea Jones,QLD,Female,250.17,0.0,Auctor Inc.,lawn and garden s...,c,1.47,0.0,9615.0,47.0,992.0,490.0,180.0,1190.0,0.8,950.0,2.4
4606,90173050473,2022-03-14,7,511685,Andrea Jones,QLD,Female,108.28,0.0,Lobortis Class In...,lawn and garden s...,c,2.49,0.0,9615.0,47.0,992.0,490.0,180.0,1190.0,0.8,950.0,2.4


In [149]:
all_combined.write.mode('overwrite').parquet("../../data/curated/all_data_combined.parquet")

In [150]:
combined = spark.read.parquet("../../data/curated/all_data_combined.parquet")

combined.limit(10)

consumer_postcode,merchant_abn,order_datetime,user_id,consumer_id,consumer_name,consumer_state,consumer_gender,transaction_dollar_value_$AUD,consumer_fraud_probability_%,merchant_name,merchant_description,merchant_revenue_level,merchant_take_rate_%,merchant_fraud_probability_%,consumer_postcode_estimated_population,consumer_postcode_median_age,consumer_postcode_median_mortgage_repay_monthly,consumer_postcode_median_totl_prsnal_inc_weekly,consumer_postcode_median_rent_weekly,consumer_postcode_median_tot_fam_inc_weekly,consumer_postcode_avg_num_psns_per_bedroom,consumer_postcode_median_tot_hhd_inc_weekly,consumer_postcode_avg_household_size
5074,92980848589,2022-08-30,44,564558,Michael Johnson,SA,Undisclosed,479.0,0.0,Maecenas Industries,"books, periodical...",a,6.86,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,58454491168,2022-08-24,44,564558,Michael Johnson,SA,Undisclosed,69.57,0.0,Diam At Foundation,computer programm...,a,6.01,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,24852446429,2022-04-18,44,564558,Michael Johnson,SA,Undisclosed,12.87,0.0,Erat Vitae LLP,florists supplies...,c,2.94,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,45559085309,2021-07-07,44,564558,Michael Johnson,SA,Undisclosed,48.19,0.0,A Auctor Non Corp...,shoe shops,a,5.58,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,58392414752,2022-09-26,44,564558,Michael Johnson,SA,Undisclosed,2.76,0.0,Mattis Ornare Lec...,florists supplies...,c,1.47,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,81219314324,2022-10-08,44,564558,Michael Johnson,SA,Undisclosed,120.55,0.0,Faucibus Leo Ltd,"gift, card, novel...",a,6.35,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,34179569263,2021-12-05,44,564558,Michael Johnson,SA,Undisclosed,816.97,0.0,Sodales At LLC,music shops - mus...,b,4.72,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,96566672398,2021-04-15,44,564558,Michael Johnson,SA,Undisclosed,81.52,0.0,Odio Tristique Ph...,"opticians, optica...",c,2.05,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,52838486584,2021-04-12,44,564558,Michael Johnson,SA,Undisclosed,1252.13,0.0,Interdum PC,tent and awning s...,a,6.19,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4
5074,32530636640,2022-03-25,44,564558,Michael Johnson,SA,Undisclosed,10.91,0.0,Hendrerit Foundation,tent and awning s...,a,5.6,0.0,21882.0,40.0,1690.0,721.0,335.0,1892.0,0.8,1427.0,2.4


In [151]:
spark.stop()